In [61]:
import pandas as pd
import numpy as np
import re

def is_row_empty_or_spaces(row):
    for val in row:
        if pd.isna(val):
            continue
        if isinstance(val, str) and val.strip() == '':
            continue
        return False
    return True

def clean_sheet_name(sheet_name):
    return re.sub(r'\W+', '_', sheet_name)

file_path = r'D:\data_set.xlsx'
all_sheets = pd.read_excel(file_path, sheet_name=None)

dataframe_names = []

for sheet_name, df in all_sheets.items():
    mask = df.apply(is_row_empty_or_spaces, axis=1)
    df_cleaned = df[~mask].copy()

    df_cleaned.insert(1, 'Klasis', sheet_name)

    cleaned_name = clean_sheet_name(sheet_name)
    df_name = 'df_' + cleaned_name

    globals()[df_name] = df_cleaned

    dataframe_names.append({'Sheet Name': sheet_name, 'DataFrame Name': df_name})

    # print(f"Dataframe '{df_name}' dibuat. Baris sebelum: {len(df)}, sesudah: {len(df_cleaned)}")

# Simpan daftar dataframe ke df_all_sheets
df_all_sheets = pd.DataFrame(dataframe_names)

print("\nDataframe df_all_sheets berhasil dibuat:")




Dataframe df_all_sheets berhasil dibuat:


In [62]:
from bson import ObjectId

# Gabungkan semua dataframe jadi satu
df_all_data = pd.concat([globals()[entry['DataFrame Name']] for entry in dataframe_names], ignore_index=True)
df_all_data.reset_index(drop=True, inplace=True)


# Ganti kolom 'NO' menjadi 'ID' dan isi dengan ObjectId acak
if 'NO' in df_all_data.columns:
    df_all_data.drop(columns='NO', inplace=True)

# Tambahkan kolom 'ID' dengan ID acak seperti MongoDB
df_all_data.insert(0, '_id', [str(ObjectId()) for _ in range(len(df_all_data))])



print(f"\nDataframe df_all_data berhasil dibuat. Total baris: {len(df_all_data)}")



Dataframe df_all_data berhasil dibuat. Total baris: 383


In [65]:
# Cek apakah ada ID yang duplikat
duplicates = df_all_data[df_all_data.duplicated(subset='_id', keep=False)]

if not duplicates.empty:
    print(f"Ada {duplicates.shape[0]} baris dengan _id duplikat:")
    print(duplicates[['_id']].value_counts())
else:
    print("Tidak ada _id yang duplikat.")


Tidak ada _id yang duplikat.


In [66]:
df_all_data = df_all_data.fillna("")



In [69]:
result_json = []
for _, row in df_all_data.iterrows():
    item = {
        "_id": row["_id"],
        "Klasis": row["Klasis"],
        "Nama Gereja": row["Nama Gereja"],
        "PERSEKUTUAN": {
            "Tahun": row["PERSEKUTUAN_Tahun"],
            "Tempat": row["PERSEKUTUAN_Tempat"],
            "Hamba Tuhan Yang Memulai": row["PERSEKUTUAN_Hamba Tuhan Yang Memulai"],
            "Orang-Orang Yang Berperan": row["PERSEKUTUAN_Orang-Orang Yang Berperan"],
            "Cara Memulai": row["PERSEKUTUAN_Cara Memulai"],
            "Alasan Memulai": row["PERSEKUTUAN_Alasan Memulai"]
        },
        "POS PI": {
            "Tahun": row["POS PI_Tahun"],
            "Tempat": row["POS PI_Tempat"],
            "Hamba Tuhan Yang Memulai": row["POS PI_Orang-Orang Yang Berperan"],
            "Orang-Orang Yang Berperan": row["POS PI_Orang-Orang Yang Berperan"],
            "Cara Memulai": row["POS PI_Cara Memulai"],
            "Alasan Memulai": row["POS PI_Alasan Memulai"]
        },
        "CABANG KEBAKTIAN": {
            "Tahun": row["CABANG KEBAKTIAN_Tahun"],
            "Tempat": row["CABANG KEBAKTIAN_Tempat"],
            "Hamba Tuhan Yang Memulai": row["CABANG KEBAKTIAN_Hamba Tuhan Yang Memulai"],
            "Orang-Orang Yang Berperan": row["CABANG KEBAKTIAN_Orang-Orang Yang Berperan"],
            "Cara Memulai": row["CABANG KEBAKTIAN_Cara Memulai"],
            "Alasan Memulai": row["CABANG KEBAKTIAN_Alasan Memulai"]
        },
        "JEMAAT LOKAL": {
            "Tahun": row["JEMAAT LOKAL_Tahun"],
            "Tempat": row["JEMAAT LOKAL_Tempat"],
            "Hamba Tuhan Yang Memulai": row["JEMAAT LOKAL_Hamba Tuhan Yang Memulai"],
            "Orang-Orang Yang Berperan": row["JEMAAT LOKAL_Orang-Orang Yang Berperan"],
            "Cara Memulai": row["JEMAAT LOKAL_Cara Memulai"],
            "Alasan Memulai": row["JEMAAT LOKAL_Alasan Memulai"]
        },
        "Keterangan": row["KETERANGAN"]
    }
    result_json.append(item)

In [70]:
import json

with open("D:/data_set_history_kibaid.json", "w", encoding="utf-8") as f:
    json.dump(result_json, f, ensure_ascii=False, indent=2, default=str)
